In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
import gensim.models as Fasttext  # For FastText embeddings
from gensim.models.fasttext import load_facebook_model

In [6]:
data = pd.read_csv("D:\Mini Project\minipj\data\Suicide_Detection.csv")


In [10]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["class"], test_size=0.2)


In [11]:
tokenizer = Tokenizer(num_words=10000)  # Adjust max vocabulary size as needed
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)


In [12]:
max_length = 100  # Adjust maximum post length as needed
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding="post")
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding="post")

In [30]:
import fasttext.util
from urllib.request import urlopen

fasttext_model = fasttext.load_model(urlopen("https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz"))

TypeError: loadModel(): incompatible function arguments. The following argument types are supported:
    1. (self: fasttext_pybind.fasttext, arg0: str) -> None

Invoked with: <fasttext_pybind.fasttext object at 0x00000273E86F91F0>, <http.client.HTTPResponse object at 0x00000273F9EE16C0>